In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from mhm_dataprocessing import *

import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity

### Read data

In [2]:
mhm_fluxes = xr.open_dataset(r"W:\VUB\_main_research\mHM\mhm_outputs\mHM_Fluxes_States.nc")

#shapefile mask
be_shp = gpd.read_file(r"W:\VUB\_main_research\mHM\mhm_belgium\be_shp\be.shp")

In [3]:
be_data = clip_to_region(be_shp, mhm_fluxes)

#resample to month ends
#resample fluxes to month ends
variables = ['SM_L01','SM_L02','SM_L03','SM_Lall','SWC_L01','SWC_L02','SWC_L03','recharge']

mhm_fluxes_mon= be_data[variables].resample(time='ME').mean()

In [4]:
#select values for June
data=mhm_fluxes_mon.sel(time=mhm_fluxes_mon.time.dt.month==6)

#select a time series from a specific location
sample_ts = mhm_fluxes_mon['SM_Lall'].sel(lon=4.5, lat=50.5, method='nearest')

In [ ]:
ts_df = sample_ts.to_dataframe().drop(columns=['lat', 'lon'])

In [ ]:
fig,ax=plt.subplots(figsize=(10,2.5))
plt.plot(ts_df.index, ts_df['SM_Lall'])
plt.axhline(y=0.73, color='r', linestyle='--')

#### SMI and drought characteristics

ref: https://www.nature.com/articles/s41558-018-0138-5

$SMI_{t} = F_{T}(x_{t})$


 Represents the quantile at the soil moisture fraction value x (normalized against the saturated soil water content). x t denotes the simulated monthly soil moisture fraction at a time t and is the empirical distribution function estimated using the kernel density estimator, $f_{t}(x)$

 $f_{t}(x) = \frac{1}{nh}\sum_{k=1}^{n}K(\frac{x-x_{k}}{h})$
  

 Here, $x_{1}$, …, $x_{n}$ n represents the simulated soil moisture fraction of a given calendar month during the reference period T; n denotes the number of calendar months within a given period (that is, 30 for a 30-year period); and $K$ represents a Gaussian kernel function with a bandwidth h. The bandwidth is estimated with GridSearch.
 
$K(x, x_{k}) = \frac{1}{\sqrt{2\pi h^{2}}}\exp(\frac{(x-x_{k})^{2}}{2h^{2}})$

A cell at time t is under drought when SMI t  < $\tau$. Here, $\tau$ denotes that the soil water content in this cell is less than the values occurring $\tau$ × 100% of the time. In this study, τ is set to 0.2
 

  
### Bandwidth selection
Find the optimal bandwidth h for the distribution function.
GridsearchCV

-- --
Alternatively using Silverman's rule (has limitations)


https://towardsdatascience.com/bounded-kernel-density-estimation-2082dff3f47f

Silverman's rule of thumb, optimal when the underlying density being estimated is Gaussian

$h = \sigma * (\frac{4}{3n})^{-0.2}$

#### Kernel Density Estimation (Example)

In [ ]:
#GridSearchCV to find the optimal bandwidth
#extract values
soil_moisture = ts_df.values[:,0]
# Define a range of bandwidths to test
bandwidths =np.linspace(0.001, 0.9, 500)

# Perform cross-validated grid search for bandwidth h
grid = GridSearchCV(KernelDensity(kernel='gaussian'),
                    {'bandwidth': bandwidths},
                    cv=5)  # 5-fold cross-validation
grid.fit(soil_moisture[:, None])

# Optimal bandwidth
optimal_h = grid.best_params_['bandwidth']
print(f"Optimal bandwidth: {optimal_h}")

#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# kernel density estimation
kde = KernelDensity(bandwidth=optimal_h, kernel='gaussian')
kde.fit(soil_moisture[:, None]) #fit the KDE model to the data

#plot the kde
x = np.linspace(soil_moisture.min() * 0.8, soil_moisture.max() * 1.1, 1000) 
#x used for evaluating the kernel density estimation (KDE) does not have to be the same as the original data points.
#  Instead, x is typically chosen as a smooth, evenly spaced range of values that covers the domain of the data.
#  This ensures that the KDE curve is displayed smoothly over the range of interest.
# x is for the visualization of the KDE, not for determining the underlying probabilities of your data

#evaluate the KDE at the x values
#range of soil moisture values
logprob = kde.score_samples(x[:, None]) #log of the probability density function

#probability density function
prob_density = np.exp(logprob)

#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////


#plot the KDE
fig, ax = plt.subplots(figsize=(10, 3.5))
plt.fill_between(x, prob_density, alpha=0.5)
#plot vertical lines (|) representing the soil moisture values, with height equal to -0.01
plt.plot(soil_moisture, np.full_like(soil_moisture, -0.01), '|b', markeredgewidth=0.7)
plt.xlabel('sm')
plt.ylabel('Probability Density')

### Cumulative distribution function

Integrate the probability distribution function to obtain the CDF

In [ ]:
# Compute the CDF by integrating the PDF
cdf = np.cumsum(prob_density) * (x[1] - x[0])

In [ ]:
### calculare the probability of soil moisture being below 0.71
cdf_071 = np.interp(0.732, x, cdf)

#calculate the sm value corresponding to 0.2 cumulative probability
sm_02 = np.interp(0.2, cdf, x)
sm_02

In [ ]:
# Plot the PDF and CDF
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(x, prob_density, label='PDF')
plt.axvline(sm_02, color='red', lw=0.5, label='20%')
plt.xlabel('Soil Moisture')
plt.ylabel('Density')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(x, cdf, label='CDF', color='orange')
plt.axvline(sm_02, color='red', lw=0.5, label='20%')
#fill between the x values up to the threshold value
plt.fill_between(x, cdf, where=(x <= sm_02), color='red', alpha=0.)

plt.xlabel('Soil Moisture')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.tight_layout()
plt.show()

### cdf for entire dataset

In [ ]:
summer_months = [6,7,8]
#select values for summer months
data_summer = mhm_fluxes_mon.sel(time=mhm_fluxes_mon.time.dt.month.isin(summer_months))

#average soil moisture for summer months for each year
data_summer_avg = data_summer.groupby('time.year').mean()

In [ ]:
data_summer_avg['SM_L02'][5].plot(vmin=0.6, vmax=0.99, cmap='viridis_r')

In [ ]:
#loop through all the cells and calculate the cdf for each cell
cdf_values = []

for i in range(0, len(data_summer_avg.lon[0:10])):
    for j in range(0, len(data_summer_avg.lat[0:10])):
        sample_ts = data_summer_avg['SM_Lall'].sel(lon=data_summer_avg.lon[i], lat=data_summer_avg.lat[j], method='nearest')
        ts_df = sample_ts.to_dataframe().drop(columns=['lat', 'lon'])
        soil_moisture = ts_df.values[:,0]
        x = np.linspace(soil_moisture.min() * 0.8, soil_moisture.max() * 1.1, 1000) 
        #compute optimal bandwidth
        optimal_h = compute_optimal_h(soil_moisture)
        # kernel density estimation
        kde = KernelDensity(bandwidth=optimal_h, kernel='gaussian')
        if np.isnan(soil_moisture).all():
            cdf_values.append(np.nan)
            continue

        kde.fit(soil_moisture[:, None])
        logprob = kde.score_samples(x[:, None])
        prob_density = np.exp(logprob)
        cdf = np.cumsum(prob_density) * (x[1] - x[0])
        cdf_values.append(cdf)

In [ ]:
import numpy as np
from sklearn.neighbors import KernelDensity
import xarray as xr

# Function to compute CDF for a single grid cell
def compute_cdf_for_cell(soil_moisture_timeseries, optimal_h):
    # Ensure the timeseries is a 1D array
    soil_moisture_timeseries = soil_moisture_timeseries.flatten()

    # Remove NaN values from the timeseries
    soil_moisture_timeseries = soil_moisture_timeseries[~np.isnan(soil_moisture_timeseries)]

    # If the timeseries is empty after removing NaNs, return NaN array for that cell
    if len(soil_moisture_timeseries) == 0:
        return np.full(1000, np.nan)

    # Fit Kernel Density Estimation (KDE) to the timeseries data
    kde = KernelDensity(bandwidth=optimal_h, kernel='gaussian')
    kde.fit(soil_moisture_timeseries[:, None])  # Fit KDE to the soil moisture timeseries

    # Generate range of x values (for CDF computation)
    x = np.linspace(soil_moisture_timeseries.min() * 0.8, soil_moisture_timeseries.max() * 1.1, 1000)

    # Evaluate the KDE at the x values (log-probabilities)
    logprob = kde.score_samples(x[:, None])

    # Convert log-probabilities to probability density function (PDF)
    prob_density = np.exp(logprob)

    # Compute CDF by integrating the PDF using the trapezoidal rule
    cdf = np.cumsum(prob_density) * (x[1] - x[0])

    # Return the full CDF array for the grid cell
    return cdf

# Main function to compute CDF for all grid cells
def compute_cdf(dataset, optimal_h, month):
    # Filter dataset for the given month
    data_month = dataset.sel(time=dataset.time.dt.month == month)

    # Impute missing values using forward fill or interpolation (if necessary)
    #data_month = data_month.fillna(method='ffill', dim='time')

    # Extract soil moisture timeseries for each grid cell
    soil_moisture_timeseries = data_month.values

    # Apply the compute_cdf_for_cell function across each grid cell
    cdf_values = xr.apply_ufunc(
        compute_cdf_for_cell,  # Function to apply
        soil_moisture_timeseries,  # Soil moisture time series for each grid cell
        optimal_h,  # The bandwidth for KDE
        input_core_dims=[['time'], []],  # Apply across the 'time' dimension (each time series per grid cell)
        vectorize=True,  # Vectorize to process all grid cells
        output_dtypes=[float],  # Output will be an array of CDF values for each grid cell
        output_sizes={'x': 1000}  # Specify the output size for the CDF (1000 points)
    )

    return cdf_values

In [ ]:
# Example usage: Compute the CDF for the 6th month (June) with the given optimal bandwidth (e.g., optimal_h=0.2)
cdf_values = compute_cdf(mhm_fluxes_mon['SM_Lall'], optimal_h=0.2, month=6)